# Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

#Variables

In [2]:
SHUFFLE_BUFFER_SIZE = 1000
BATCH_SIZE = 128
LR= 0.1
NUM_EPOCHS=100

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship','truck']

# Loading Dataset

In [3]:
def load_and_prep_cifar(batch_size, shuffle_buffer_size):
  #train, test = tfds.load('cifar10', split=['train','test'], as_supervised=True)
  train_ds, test_ds= tfds.load('cifar10', split=['train','test'] , as_supervised=True)

  def preprocessing_func(img, label):
    img = tf.cast(img, tf.float32)
    img = (img/128)-1
    label = tf.one_hot(label, depth = 10)
    return img, label


  train_ds = train_ds.map(lambda img, label: preprocessing_func(img,label))
  test_ds = test_ds.map(lambda img, label: preprocessing_func(img,label))
  train_ds = train_ds.shuffle(shuffle_buffer_size).batch(batch_size).prefetch(2)
  test_ds = test_ds.batch(batch_size).prefetch(2)
  return train_ds, test_ds

train_ds, test_ds = load_and_prep_cifar(batch_size=BATCH_SIZE, shuffle_buffer_size=SHUFFLE_BUFFER_SIZE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteMP3MZS/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteMP3MZS/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


# Model

In [4]:
def create_my_cifar_cnn(name= 'my_cnn'):
  inputs= tf.keras.Input(shape= (32,32,3),dtype=tf.float32)

  cnn_layer_1_1= tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3), activation = 'relu',padding='same')
  x= cnn_layer_1_1(inputs)#shape: [batch_size, 32,32,16]

  cnn_layer_1_2= tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3), activation = 'relu',padding='same')
  x= cnn_layer_1_2(x)#shape: [batch_size, 32,32,16]

  cnn_layer_pooling1 = tf.keras.layers.MaxPool2D()
  x= cnn_layer_pooling1(x)#shape: [batch_size, 16,16,16]

  cnn_layer_2_1= tf.keras.layers.Conv2D(filters = 32, kernel_size=(3,3), activation = 'relu',padding='same')
  x= cnn_layer_2_1(x)#shape: [batch_size, 16,16,32]

  cnn_layer_2_2= tf.keras.layers.Conv2D(filters = 32, kernel_size=(3,3), activation = 'relu',padding='same')
  x= cnn_layer_2_2(x)#shape: [batch_size, 16,16,32]

  cnn_layer_pooling2 = tf.keras.layers.MaxPool2D()
  x= cnn_layer_pooling2(x)#shape: [batch_size, 8,8,32]

  cnn_layer_3_1= tf.keras.layers.Conv2D(filters = 64, kernel_size=(3,3), activation = 'relu',padding='same')
  x= cnn_layer_3_1(x)#shape: [batch_size, 8,8,64]

  cnn_layer_3_2= tf.keras.layers.Conv2D(filters = 64, kernel_size=(3,3), activation = 'relu',padding='same')
  x= cnn_layer_3_2(x)#shape: [batch_size, 8,8,64]

  cnn_layer_flatten = tf.keras.layers.Flatten()
  x= cnn_layer_flatten(x)# shape: [batch_size, 4096]

  dense_layer_1 = tf.keras.layers.Dense(64, activation = 'relu')
  x= dense_layer_1(x)#shape: [batch_size, 64]

  dense_layer_2 = tf.keras.layers.Dense(32, activation = 'relu')
  x= dense_layer_2(x)#shape: [batch_size, 32]

  output_layer = tf.keras.layers.Dense(10, activation = 'softmax')
  output = output_layer(x)#shape: [batch_size, 10]

  model = tf.keras.Model(inputs , output,name=name)

  return model

#Train_Model

In [5]:
def train_cifar_10(model, train_ds, test_ds, num_epochs, lr):
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")

    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []

    @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = model(images)
            loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

    @tf.function
    def test_step(images, labels):
        predictions = model(images)
        t_loss = loss_object(labels, predictions)

        test_loss(t_loss)
        test_accuracy(labels, predictions)

    for epoch in range(num_epochs):
        # Training loop
        for images, labels in train_ds:
            train_step(images, labels)

        # Testing loop
        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)

        # Save metrics for plotting
        train_losses.append(train_loss.result().numpy())
        train_accuracies.append(train_accuracy.result().numpy())
        test_losses.append(test_loss.result().numpy())
        test_accuracies.append(test_accuracy.result().numpy())

        template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.2f}%, Test Loss: {:.4f}, Test Accuracy: {:.2f}%'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              train_accuracy.result() * 100,
                              test_loss.result(),
                              test_accuracy.result() * 100))

        # Reset the metrics for the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

    # Plotting
    plt.figure(figsize=(12, 4))

    # Plot training loss
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train')
    plt.plot(test_losses, label='Test')
    plt.title('Training and Testing Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training accuracy
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train')
    plt.plot(test_accuracies, label='Test')
    plt.title('Training and Testing Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

#Calls

In [ ]:
# Train the model using the defined function
model = create_my_cifar_cnn()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss='categorical_crossentropy', metrics=['accuracy', 'categorical_accuracy'])

train_cifar_10(model, train_ds, test_ds, NUM_EPOCHS, LR)

# Evaluate the model on the test set
test_metrics = model.evaluate(test_ds)
test_loss = test_metrics[0]
test_accuracy = test_metrics[1]

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1, Loss: 4032.1582, Accuracy: 10.10%, Test Loss: 2.3114, Test Accuracy: 10.00%
Epoch 2, Loss: 2.3083, Accuracy: 10.05%, Test Loss: 2.3091, Test Accuracy: 10.00%
Epoch 3, Loss: 2.3081, Accuracy: 9.88%, Test Loss: 2.3127, Test Accuracy: 10.00%
Epoch 4, Loss: 2.3087, Accuracy: 10.16%, Test Loss: 2.3091, Test Accuracy: 10.00%
Epoch 5, Loss: 2.3090, Accuracy: 9.94%, Test Loss: 2.3097, Test Accuracy: 10.00%
Epoch 6, Loss: 2.3089, Accuracy: 10.18%, Test Loss: 2.3095, Test Accuracy: 10.00%
Epoch 7, Loss: 2.3085, Accuracy: 10.17%, Test Loss: 2.3129, Test Accuracy: 10.00%
Epoch 8, Loss: 2.3088, Accuracy: 10.12%, Test Loss: 2.3103, Test Accuracy: 10.00%
Epoch 9, Loss: 2.3092, Accuracy: 9.93%, Test Loss: 2.3146, Test Accuracy: 10.00%
Epoch 10, Loss: 2.3099, Accuracy: 10.00%, Test Loss: 2.3092, Test Accuracy: 10.00%
Epoch 11, Loss: 2.3088, Accuracy: 10.28%, Test Loss: 2.3096, Test Accuracy: 10.00%
Epoch 12, Loss: 2.3094, Accuracy: 10.05%, Test Loss: 2.3192, Test Accuracy: 10.00%
Epoch 13, Los